# Assignment 12 - Natural Language Processing

##### 1. Load the tokenized Paradise Lost from the Gutenberg Corpus in NLTK.

- Stem or lemmatize the words and find counts.
- Exclude stop words and make sure you are including words of all capitalizations in your count. If there are any meaningless “words” (“thus” and single letters, etc.) that are produced in your list of top words, alter your logic to exclude them.

In [9]:
!pip3 install nltk
import nltk
nltk.download('gutenberg')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('stopwords')

Defaulting to user installation because normal site-packages is not writeable


[nltk_data] Downloading package gutenberg to
[nltk_data]     /Users/connorlove/nltk_data...
[nltk_data]   Package gutenberg is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/connorlove/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/connorlove/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/connorlove/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [21]:
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.probability import FreqDist

pl = nltk.corpus.gutenberg.words('milton-paradise.txt')
sw = set(stopwords.words('english'))
other_stopwords = {'thou', 'thy', 'thee', 'thus', 'yet', 'though', 'may'}
sw.update(other_stopwords)

filtered_pl = [word.lower() for word in pl if word.isalnum() and word.lower() not in sw]

lemmatizer = WordNetLemmatizer()
paradise_lost_lemmatized = [lemmatizer.lemmatize(word) for word in filtered_pl]

pl_word_freq = FreqDist(paradise_lost_lemmatized)

print("Top 20 words:")
for word, freq in pl_word_freq.most_common(20):
    print(word)

Top 20 words:
heaven
god
shall
earth
u
man
first
day
high
one
son
far
death
power
great
till
like
world
hell
thing


- Select the top 20 words and create a bar chart.

- Specify why you chose stemming or lemmatization.

I chose lemmatization since it's a more complex process—and is thusly, more accurate—than stemming. Although stemming will suffice for more simple requests, lemmatization, critically, considers the CONTEXT of a word where stemming does not, before returning its lemma.

#### 2. Perform Vader Sentiment Analysis on the book.

- Find the 5 most negative, 5 most positive, and 5 most neutral sentences in Paradise Lost. This may take a while to run, so you can always start with a small subset of the data (100 sentences) and then once your code works as expected, expand it to the whole book and let it run.

#### 3. Explain your findings from the previous question. Are the sentences and their sentiment analysis scores correct? Explain why or why not.

#### Extra credit: create a word cloud for your results from question 1 (see chapter 21 in your book for how to do this).